In [ ]:
cd ..

In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
import pandas as pd
import os
import pickle
import sqlalchemy
from sklearn.ensemble import RandomForestRegressor

from project_real_estate.models.preprocessor import PropertyPreprocessor
from project_real_estate.models.rent_estimator import SKLearnRentEstimator

In [ ]:
sql_rentals = "SELECT * FROM rentals LIMIT 10;"
sql_sales = "SELECT * FROM sales LIMIT 10;"
uri = os.environ["DB_URI"]

engine = sqlalchemy.create_engine(uri)
rentals_data = pd.read_sql(sql=sql_rentals, con=engine)
sales_data = pd.read_sql(sql=sql_sales, con=engine)

# Model

In [ ]:
preprocessor = PropertyPreprocessor()
estimator = RandomForestRegressor(n_estimators=10, max_depth=10)
model = SKLearnRentEstimator(preprocessor, estimator)

In [ ]:
model.fit(rent_data)

In [ ]:
model.predict(sales_data)